<a href="https://colab.research.google.com/github/BorlotW/WebapiSpotify/blob/main/WebApiSpotify.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

##### Teste: Consumo de Dados da Api do Spotify para analises sobre artistas e músicas.

In [3]:
!pip install -q findspark
!pip install -q unidecode
!pip install -q pyspark
import findspark
findspark.init()

import requests
import json
import unidecode
from pyspark.sql import SparkSession

spark = SparkSession.builder \
      .master("local[1]") \
      .appName("Teste_Spark_Autoglass_candidatos") \
      .getOrCreate()

#### Get Token

In [134]:
client_id = ""
client_secret = ""

In [135]:
def GetToken(client_id,client_secret):
  url = 'https://accounts.spotify.com/api/token'
  data = {'grant_type': 'client_credentials'}
  auth = (client_id, client_secret)
  response = requests.post(url, data=data, auth=auth)
  token = response.json().get('access_token')
  return token

def GetTokenPrivate(client_id,client_secret):
  redirect_uri = 'https://www.example.com/oidc_callback'
  scopes = 'user-library-read playlist-read-private'
  authorize_url = 'https://accounts.spotify.com/authorize'
  token_url = 'https://accounts.spotify.com/api/token'
  auth_params = {
      'client_id': client_id,
      'response_type': 'code',
      'redirect_uri': redirect_uri,
      'scope': scopes,
  }
  auth_response = requests.get(authorize_url, params=auth_params)
  print("Por favor, acesse esta URL para autorizar a aplicação:", auth_response.url)
  authorization_code = input("Por favor, insira o código de autorização: ")
  token_data = {
      'grant_type': 'authorization_code',
      'code': authorization_code,
      'redirect_uri': redirect_uri,
      'client_id': client_id,
      'client_secret': client_secret,
  }
  response = requests.post(token_url, data=token_data)
  token = response.json()['access_token']
  return token

#### Funções

### Busca no Spotify generos de Artistas

In [6]:
def BuscaArtist(api_token, genre, offset=0):
  url = 'https://api.spotify.com/v1/search'
  limit = 50
  headers = {'Authorization': f'Bearer {access_token}'}
  params = {
    'q': f'genre:{genre}',
    'type': 'artist',
    'limit': limit,
    'offset': offset,
    'market' : 'BR',
  }

  response = requests.get(url, params=params, headers=headers)
  # Verificar se a requisição foi bem-sucedida
  if response.status_code == 200:
    artists = response.json().get('artists', {}).get('items', [])
    if len(artists) == limit:
      next_offset = offset + limit
      more_artists = BuscaArtist(api_token, genre, offset=next_offset)
      artists.extend(more_artists)
  return artists

##### Extraindo os artistas pelos generos ("Rock Nacional", "Piseiro/Arrocha" e "Pop Internacional")

In [7]:
access_token = GetToken(client_id, client_secret)

genres = ['"Rock Nacional"', '"Piseiro/Arrocha"', '"Pop Internacional"']
i = 0
for genre in genres:
  if i == 0:
    lt = BuscaArtist(access_token, genre)
    i=1
    print(f'Extraindo dados de {genre}, com um total de {len(lt)} registros')
  else:
    more_genre = BuscaArtist(access_token, genre)
    lt.extend(more_genre)
    i=i+1
    print(f'Extraindo dados de {genre}, com um total de {len(more_genre)} registros')

print(f'Extraindo {len(lt)} registros')

Extraindo dados de "Rock Nacional", com um total de 253 registros
Extraindo dados de "Piseiro/Arrocha", com um total de 6 registros
Extraindo dados de "Pop Internacional", com um total de 0 registros
Extraindo 259 registros


In [370]:
Artist = spark.createDataFrame(lt)

In [9]:
Artist.count()

259

#### Armazene os dados em formato parquet, particionando por artista;

In [10]:
Artist.write.partitionBy("id").mode("overwrite").format("parquet").save("/content/parquet/artist")

##### Lendo dados através dos arquivos parquet

In [11]:
file_path = "/content/parquet/artist"
parquet = spark.read.parquet(file_path)
Dados = parquet.filter(parquet['name'] == 'Pez').show()

+--------------------+--------------------+--------------------+--------------------+--------------------+----+----------+------+--------------------+--------------------+
|       external_urls|           followers|              genres|                href|              images|name|popularity|  type|                 uri|                  id|
+--------------------+--------------------+--------------------+--------------------+--------------------+----+----------+------+--------------------+--------------------+
|{spotify -> https...|{total -> 33474, ...|[argentine indie,...|https://api.spoti...|[{width -> 640, u...| Pez|        29|artist|spotify:artist:00...|00demsNI1p5KmzWGl...|
+--------------------+--------------------+--------------------+--------------------+--------------------+----+----------+------+--------------------+--------------------+



### Crie um dataframe com o endpoint "Get Artist"

In [12]:
access_token = GetToken(client_id, client_secret)

# Função para obter informações sobre um artista pelo ID
def GetArtist(artist_id, token):
    url = f'https://api.spotify.com/v1/artists/{artist_id}'
    headers = {'Authorization': f'Bearer {token}'}

    response = requests.get(url, headers=headers)
    data = response.json()

    return data

In [ ]:
for row in lt:
  print(row['id'])

In [ ]:
import pandas as pd

schema = ['id','name','popularity','followers']
df = pd.DataFrame(columns = schema)

access_token = GetToken(client_id,client_secret)
lt = Artist.select('id').collect()
print('Iniciando Extração...')
for row in lt:
  lt1 = GetArtist(row['id'], access_token)
  df1 = pd.DataFrame([ [lt1["id"], lt1.get("name"), lt1.get("popularity"), lt1.get("followers").get("total")] ], columns = schema)
  df = pd.concat([df,df1], ignore_index=True)
  # print(row['id'])
print('Fim da Extração, com ')

df_artists = spark.createDataFrame(df)
df_artists.show()

### Crie um dataframe com o endpoint "Get Artist's Albums" e traga as músicas dos álbuns que estão no endpoint "Get Album Tracks".

In [43]:
def GetArtistAlbums(access_token, id, offset=0):
  url = f"https://api.spotify.com/v1/artists/{id}/albums"
  limit = 50
  headers = {'Authorization': f'Bearer {access_token}'}
  params = {
    'limit': limit,
    'offset': offset,
    'market' : 'BR',
  }

  response = requests.get(url, params=params, headers=headers)
  if response.status_code == 200:
    albums = response.json()["items"]
    if len(albums) == limit:
      next_offset = offset + limit
      more_albums = GetArtistAlbums(access_token, id, offset=next_offset)
      albums.extend(more_albums)
    return albums
  else:
    print(f"Erro ao obter os álbuns do artista. Código de status: {response.status_code}")
    return None

In [25]:
access_token = GetToken(client_id,client_secret)

lt = Artist.select('id').collect()
i=0
for row in lt:
  if i == 0:
    lt1 = GetArtistAlbums(access_token, row['id'])
    i=i+1
  else:
    lt2 = GetArtistAlbums(access_token, row['id'])
    lt1 = lt1+lt2
    i=i+1

print(f'Fim da Extração, com {i} extrações')

Fim da Extração, com 259 extrações


In [27]:
df_albums = spark.createDataFrame(lt1)

In [ ]:
df_albums.show()

In [66]:
def GetTracks(access_token, id, offset=0):
  url = f"https://api.spotify.com/v1/albums/{id}/tracks"
  limit = 50
  headers = {'Authorization': f'Bearer {access_token}'}
  params = {
    'limit': limit,
    'offset': offset,
    'market' : 'BR',
  }

  response = requests.get(url, params=params, headers=headers)
  if response.status_code == 200:
    tracks = response.json()["items"]
    if len(tracks) == limit:
      next_offset = offset + limit
      more_albums = GetTracks(access_token, id, offset=next_offset)
      tracks.extend(more_albums)
    return tracks
  else:
    print(f"Erro ao obter as tracks do Album. Código de status: {response.status_code}")
    return None

In [ ]:
access_token = GetToken(client_id,client_secret)
i=0
lt = df_albums.select('id').collect()
for row in lt:
  if i == 0:
    tr1 = GetTracks(access_token, row['id'])
    i=i+1
  else:
    tr2 = GetTracks(access_token, row['id'])
    tr1 = tr1+tr2
    i=i+1

In [71]:
df_tracks = spark.createDataFrame(tr1)

### Criando DataFrame My play List

In [101]:
def GetMyPlaylist (access_token):
  url = "https://api.spotify.com/v1/me/playlists"
  headers = {'Authorization': f'Bearer {access_token}'}
  response = requests.request("GET", url, headers=headers)
  return response.json()

In [102]:
accces_token = GetTokenPrivate(client_id,client_secret)

Por favor, acesse esta URL para autorizar a aplicação: https://accounts.spotify.com/en/login?continue=https%3A%2F%2Faccounts.spotify.com%2Fauthorize%3Fscope%3Duser-library-read%2Bplaylist-read-private%26response_type%3Dcode%26redirect_uri%3Dhttps%253A%252F%252Fwww.example.com%252Foidc_callback%26client_id%3D0297e6853ee643fbb74ed9d45a4f90f6
Por favor, insira o código de autorização: AQBysLLqBcca4qRU8ZNN08LS9Hi6VmPfTg986fUDeRyzhkf_djM5arK9OSvmLjzyg3_r-GTZI2DCcAEWKKOvz-jxIRtUYaS0zw07tqVnO59Mco4LliQ1YoDH7XEd3XT8wqvfPKFv4UczNSG2BtOb-9kfF0a6QReqjBJIqYeyYUIhgpYFIXFrdptaJs75XwKJY3zCTgYxY3pEnMNI_O4LA4qre_zJs_HWK05Q187U7nuv6YQ


In [141]:
playlists_data = GetMyPlaylist(accces_token)

In [144]:
playlists_info = [(playlist["id"],playlist["name"],playlist["description"],playlist.get("tracks").get("total")) for playlist in playlists_data["items"]]
df_playlist = spark.createDataFrame(playlists_info, ["id", "name", "description","tracks"])
df_playlist.show()

+--------------------+---------------+--------------------+------+
|                  id|           name|         description|tracks|
+--------------------+---------------+--------------------+------+
|37i9dQZF1DWWr5uie...|Trapperz Brasil|o melhor do trap ...|    50|
|37i9dQZF1DX0FOF1I...|     Top Brasil|Manu, Simone Mend...|    50|
|37i9dQZF1E38NFXWH...|    Daily Mix 2|Henrique & Julian...|    50|
|7wvzE4xu5HsOCaaFj...|               |                    |    22|
+--------------------+---------------+--------------------+------+



### Criando uma tabela temporária em PySpark a partir do DataFrame de músicas dos artistas


In [ ]:
df_tracks.createOrReplaceTempView("tracks")

consulta_sql = "SELECT * FROM tracks"
result = spark.sql(consulta_sql)
result.show(truncate=False)

### Criando uma tabela temporária em Pyspark a partir do Dataframe de Playlists


In [113]:
df_playlist.createOrReplaceTempView("playlist")

### Execute uma consulta SQL que retorna os artistas do endpoint "Get Artist" que estão nas Playlists, ordenados por ordem alfabética.


In [120]:
# result = spark.sql('select * from playlist')
# result.show()

df_artists.sort('name').show()

+--------------------+--------------------+----------+---------+
|                  id|                name|popularity|followers|
+--------------------+--------------------+----------+---------+
|4cz8toptnAtxffn7g...|              14 Bis|        44|   319250|
|3vFybByEN1a5cjYoa...|              AN4NDA|         0|        0|
|0RZvJL3AicNni7cis...|        Adrián Otero|        19|    13015|
|0oIxiHRBpqq0VJe3w...|            Aeroblus|        14|     8078|
|78RO0kh8OAJf6vMKV...|        Afonso Nigro|        17|     4634|
|1DAJ5jfLrqvB6DU52...|                Alas|         3|     1330|
|2MvoqSSTigBbfXWoS...|    Alejandro Medina|        13|     4663|
|6Qv7PzO62iPNtE9jN...|         Alma Y Vida|        16|    12347|
|7x2a9uyqlWbE9Lwco...|            Almendra|        49|   371706|
|3SF82To29Y7SnNbgo...|               AnamA|         0|      231|
|1OnaDJeU2PxDBZZFk...|Anderson & Vei da...|        45|   249075|
|2GD7vRCBhQZKVChMM...|              Anfear|         1|      640|
|6HGVx8BKrpH04jAQW...|   

In [123]:
df_artists.createOrReplaceTempView("tabela_temporaria_artistas")

consulta_sql = "SELECT * FROM tabela_temporaria_artistas ORDER BY name"
result = spark.sql(consulta_sql)
result.show()

+--------------------+--------------------+----------+---------+
|                  id|                name|popularity|followers|
+--------------------+--------------------+----------+---------+
|4cz8toptnAtxffn7g...|              14 Bis|        44|   319250|
|3vFybByEN1a5cjYoa...|              AN4NDA|         0|        0|
|0RZvJL3AicNni7cis...|        Adrián Otero|        19|    13015|
|0oIxiHRBpqq0VJe3w...|            Aeroblus|        14|     8078|
|78RO0kh8OAJf6vMKV...|        Afonso Nigro|        17|     4634|
|1DAJ5jfLrqvB6DU52...|                Alas|         3|     1330|
|2MvoqSSTigBbfXWoS...|    Alejandro Medina|        13|     4663|
|6Qv7PzO62iPNtE9jN...|         Alma Y Vida|        16|    12347|
|7x2a9uyqlWbE9Lwco...|            Almendra|        49|   371706|
|3SF82To29Y7SnNbgo...|               AnamA|         0|      231|
|1OnaDJeU2PxDBZZFk...|Anderson & Vei da...|        45|   249075|
|2GD7vRCBhQZKVChMM...|              Anfear|         1|      640|
|6HGVx8BKrpH04jAQW...|   

In [124]:
df_playlist.show()

+--------------------+---------------+--------------------+
|                  id|           name|         description|
+--------------------+---------------+--------------------+
|37i9dQZF1DWWr5uie...|Trapperz Brasil|o melhor do trap ...|
|37i9dQZF1DX0FOF1I...|     Top Brasil|Manu, Simone Mend...|
|37i9dQZF1E38NFXWH...|    Daily Mix 2|Henrique & Julian...|
|7wvzE4xu5HsOCaaFj...|               |                    |
+--------------------+---------------+--------------------+



### 1

In [352]:
# "https://api.spotify.com/v1/playlists/37i9dQZF1DWWr5uieiPUVM/tracks"

def GetTracksPlaylist(access_token, idPlaylist, offset=0):
  url = f"https://api.spotify.com/v1/playlists/{idPlaylist}/tracks"
  limit = 50
  headers = {'Authorization': f'Bearer {access_token}'}
  params = {
    'limit': limit,
    'offset': offset,
    'market' : 'BR',
    'fields': 'items(track(id,name,artists(id,name)))',
  }

  response = requests.get(url, params=params, headers=headers)
  return response.json()

In [342]:
access_token = GetToken(client_id,client_secret)

# GetTracksPlaylist(access_token, id).json()

In [354]:
from pyspark.sql.functions import explode, explode_outer, col

lt = df_playlist.select('id').collect()
lt
i=0
for row in lt:
  if i == 0:
    lt1 = GetTracksPlaylist(access_token, row['id'])
    df_tracks_playlist = spark.createDataFrame(lt1['items'])
    i=i+1
  else:
    lt2 = GetTracksPlaylist(access_token, row['id'])
    more_tracks_playlist = spark.createDataFrame(lt2['items'])
    df_tracks_playlist = df_tracks_playlist.union(more_tracks_playlist)
    i=i+1

print(f'Fim da Extração, com {i} extrações')

lt1 = GetTracksPlaylist(access_token, '37i9dQZF1DWWr5uieiPUVM')
lt = lt1.get('items')

df_tracks_playlist = df_tracks_playlist.select(
    col("track.id").alias("track_id"),
    col("track.name").alias("track_name"),
    explode_outer("track.artists").alias("artists")
)

df_tracks_playlist = df_tracks_playlist.select(
    col("track_id"),
    col("track_name"),
    col("artists.name").alias("artist_name"),
    col("artists.id").alias("artist_id")
)

# df_tracks_playlist.show(truncate=False)

Fim da Extração, com 4 extrações


In [227]:
df_playlist.show()

+--------------------+---------------+--------------------+------+
|                  id|           name|         description|tracks|
+--------------------+---------------+--------------------+------+
|37i9dQZF1DWWr5uie...|Trapperz Brasil|o melhor do trap ...|    50|
|37i9dQZF1DX0FOF1I...|     Top Brasil|Manu, Simone Mend...|    50|
|37i9dQZF1E38NFXWH...|    Daily Mix 2|Henrique & Julian...|    50|
|7wvzE4xu5HsOCaaFj...|               |                    |    22|
+--------------------+---------------+--------------------+------+



In [365]:
df_tracks_playlist.show(truncate=False)

+--------+----------+-----------------+----------------------+
|track_id|track_name|artist_name      |artist_id             |
+--------+----------+-----------------+----------------------+
|NULL    |NULL      |Tz da Coronel    |3lIU3RoZiHen1QXAQ3KQ9e|
|NULL    |NULL      |Dj Alle da coro  |6fGH2Qk8eKAcvEU1nHOi8x|
|NULL    |NULL      |Cúpula           |6IlN4Mij49rPYznzeOMD1F|
|NULL    |NULL      |MC Ryan SP       |75i9GaW2MJUgt4BkdUnuUY|
|NULL    |NULL      |KayBlack         |2h5Ha0ZiMFmOQD3iYcSXsy|
|NULL    |NULL      |MC Caverinha     |7tojcuXcZ7eBlQjbAc65Ee|
|NULL    |NULL      |Portugal No Beat |71LRKKvAjoc6LyNmob269Q|
|NULL    |NULL      |Vulgo FK         |27azwwkxutWL1BWMkgNIh0|
|NULL    |NULL      |Orochi           |3rfM2cGqF6DB0kUyytMkXx|
|NULL    |NULL      |Caio Luccas      |1a3fr7NdeBT4JlGj6YlbDL|
|NULL    |NULL      |Pedro Lotto      |23ot0eI6ByBW6LrlBfr2bm|
|NULL    |NULL      |Mainstreet       |25XJqeReVV38w0tR04GGBd|
|NULL    |NULL      |Galdino          |0WvTL8PecsD3VxRG

In [228]:
df_artists.show()

+--------------------+--------------------+----------+---------+
|                  id|                name|popularity|followers|
+--------------------+--------------------+----------+---------+
|1on7ZQ2pvgeQF4vmI...|   Charlie Brown Jr.|        71|  6657197|
|6tw6EpC9RgmSRZiZg...|       Legião Urbana|        65|  6097390|
|4C4kpaAdp6aKSkguw...|               Skank|        64|  2887232|
|1A5QJAC1vdhbhPE25...|             O Rappa|        63|  3529943|
|0GNq4xh8uFCyihPur...|Engenheiros Do Ha...|        62|  2350715|
|0A1oy7PC7fdzURgaL...|         Lulu Santos|        62|  1965180|
|5re6NsyXzHg6uvNim...|          Jota Quest|        61|  2252837|
|4Z0yuwHVJBROVZqFp...|     Capital Inicial|        60|  2818426|
|2euX7vCVnJy3TVEGf...|               Titãs|        60|  2127467|
|7EM9m7HOXxVgP9oEp...|Os Paralamas Do S...|        60|  2185639|
|1PwOU6fFbmaGkK3wk...|              Cazuza|        59|  1863470|
|7jrRQZg4FZq6dwpi3...|         Raul Seixas|        59|  2158561|
|4bOZtegYNmYOe3gMg...|   

### Executando uma consulta SQL que retorna os artistas do endpoint "Get Artist" que estão nas Playlists, ordenados por ordem alfabética.


In [369]:
df_artists.createOrReplaceTempView("tabela_temporaria_artistas")
# df_tracks_playlist.distinct().select('artist_id').createOrReplaceTempView("artist_pl")

df = df_tracks_playlist.select('artist_id')
df = df.distinct()
df.createOrReplaceTempView("artist_pl")

consulta_sql = "SELECT a.* FROM tabela_temporaria_artistas a inner join artist_pl on artist_pl.artist_id = a.id ORDER BY a.name"
result = spark.sql(consulta_sql)
result.show()

+--------------------+-----------+----------+---------+
|                  id|       name|popularity|followers|
+--------------------+-----------+----------+---------+
|1A5QJAC1vdhbhPE25...|    O Rappa|        63|  3529943|
|3whgFbrRxsOmYVrq3...|Paulo Pires|        57|    44896|
+--------------------+-----------+----------+---------+



### Criando um Dataframe com o resultado da consulta e salve em parquet

In [371]:
result.write.mode("overwrite").format("parquet").save("/content/parquet/resultados")